In [115]:
import networkx as nx
import pandas as pd
import numpy as np
from graph.twittergraph import TwitterGraph as tg
from graph.graph import Graph
from graph.enrongraph import EnronGraph as eg
from graph.fbgraph import FBGraph as fb
from graph.collabgraph import CollabGraph as cg
import os
import subprocess
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, ndcg_score

In [117]:
# graph = tg.rt_graph_from_json('/Users/tomfw/Downloads/DataShared/', 0)
graph = eg.load_enron_graph('/Users/tomfw/Downloads/Data/Enron/EnronDirectedWithCc_7days.mat')
# graph = fb.load_fb_graph('/Users/tomfw/Downloads/Data/Facebook/FacebookFilteredAdj_90Days_6ActiveTimes_30OutInDeg.mat')
# graph = cg.load_collab_graph('/Users/tomfw/Downloads/Data/DBLP2/citation2Filtered.mat')

In [118]:
#collab .83
#facebook .90
#enron .60
#S Africa .88

In [102]:
data_folder = '/Users/tomfw/Desktop/temp/'

In [119]:
sgs = graph.subgraphs_of_length(periods=21)
print("Made %d graphs." % len(sgs))

Made 9 graphs.


In [104]:
edges = 0
for i, sg in enumerate(sgs):
    e = sg.nx_graph.number_of_edges()
    edges += e
    print("%d: %d edges,  %d nodes" % (i, e, sg.nx_graph.number_of_nodes()))
print("\nOriginal graph edges: %d\nSum of edges  in subgraphs: %d" % (graph.nx_graph.number_of_edges(), edges))

0: 15 edges,  182 nodes
1: 50 edges,  182 nodes
2: 172 edges,  182 nodes
3: 315 edges,  182 nodes
4: 580 edges,  182 nodes
5: 713 edges,  182 nodes
6: 1044 edges,  182 nodes
7: 1055 edges,  182 nodes
8: 298 edges,  182 nodes

Original graph edges: 2097
Sum of edges  in subgraphs: 4242


In [120]:
core_nodes = []
prev_embeds = []  # subgraphs * len(core_nodes)
for _ in sgs:
    prev_embeds.append([])
for node in graph.nx_graph.nodes_iter():
    is_core = True
    for sg in sgs:
        if sg.nx_graph.degree(node) == 0:
            is_core = False
    if is_core:
        core_nodes.append(node)
            
print("Found %d core nodes." % len(core_nodes))

Found 5 core nodes.


In [121]:
def store_core_embeds(embed_dict):
    embeds = []
    for node in core_nodes:
        embeds.append(embed_dict[node])
    return embeds


def core_movement(embed_dict):
    dist = 0
    for i, node in enumerate(core_nodes):
        dist += embedding_distance(prev_embeds[i], embed_dict[node])
    return dist


def embedding_distance(x1, x2):
    d = 0
    for x, y in zip(x1, x2):
        d += (x - y) ** 2
    return np.sqrt(d)

In [123]:
line_path = '/Users/tomfw/Downloads/temporalnode2vec/lineLinux/line'
rf_path = '/Users/tomfw/Downloads/temporalnode2vec/word2vec/retrofit_word2vec_one'
n2v_path = '/Users/tomfw/Desktop/snap/examples/node2vec/node2vec'

In [ ]:
def line_command(train, output, size=128, threads=8, negative=5):
    # todo: order, rho, etc...
    command = [line_path, "-train", train, "-output",  output, "-size", str(size), "-threads", str(threads),
               "-negative", str(negative)]
    return command

def rf_command(input, output, init, beta_file, size=128, window=5, sample=0, negative=5, threads=8, beta=1):
    command = [rf_path,"-train", input, "-init", init, "-output", output,
               "-size", str(size), "-window", str(window), "-sample", str(sample),
               "-negative", str(negative), "-threads", str(threads), "-beta", str(beta), "-cbow", '0']
    return command

def n2v_command(edge_file, output, n_walks=10, walk_length=50, p=1, q=1):
    command = [n2v_path, '-i:' + edge_file, '-o:' + output, '-p:' + str(p), '-q:' + str(q),
               '-r:' + str(n_walks), '-l:' + str(walk_length), '-w', '1', '-v', '1']
    return command

In [127]:
def run_command(command):
    process = subprocess.Popen(command, stderr=subprocess.PIPE)
    err = process.communicate()
    if err[0]:
        print err

In [128]:
embed_file = data_folder + 'embeddings.txt'
walk_file = data_folder + 'walks.txt'
init_file = data_folder + 'init.txt'
beta_file = data_folder + 'betas.txt'
edge_file = data_folder + 'e_list.txt'


emb_command = rf_command(walk_file, embed_file, init_file, beta_file, beta=1)
walk_command = n2v_command(edge_file, walk_file, p=1, q=1, n_walks=10, walk_length=50)
classifier = None
pred = None
for i, sg in enumerate(sgs):
    print("Current time period: (%d/%d)" % (i + 1, len(sgs)))
    cum = graph.subgraph_within_dates(sgs[0].min_date, sg.max_date).nx_graph

    sg.save_edgelist(edge_file)
    if i == 0:
        run_command(line_command(edge_file, output=embed_file))
        sg.load_embeddings(embed_file)
        sg.save_embeddings(init_file, 128)
        prev_embeds = store_core_embeds(sg.embeddings)
    else:
        prev = sgs[i - 1]
        if i == 4:
            print("\tFit...")
            train_graph = graph.subgraph_within_dates(sg.min_date, sgs[i + 2].max_date)
            train_graph.embeddings = prev.embeddings
            train_graph.emb_cols = prev.emb_cols
            train_pairs = prev.make_pairs_with_edges(train_graph, .5, enforce_non_edge=False, enforce_has_embeddings=True)
            df_train, y_train = prev.to_dataframe(pairs=train_pairs, label_graph=train_graph)
            rf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2, random_state=0, n_jobs=-1)
            fields = prev.emb_cols
            x_train = df_train.loc[:, fields]
            classifier = rf.fit(x_train, y_train)
            print("\tModel fitted")
        if i == 6:
            print("\tTesting...")
            test_graph = graph.subgraph_within_dates(sg.min_date, sgs[i+2].max_date)
            test_graph.embeddings = prev.embeddings
            test_graph.emb_cols = prev.emb_cols
            test_pairs = prev.make_pairs_with_edges(test_graph, .5, enforce_non_edge=False, enforce_has_embeddings=True)
            df_test, y_test = prev.to_dataframe(test_pairs, label_graph=test_graph)
            fields = prev.emb_cols
            x_test = df_test.loc[:, fields]
            pred = classifier.predict_proba(x_test)
            print("Prediction made.... Done")
            break
        sg.generate_embeddings_with_prev(prev.embeddings, 128)
        run_command(walk_command)
        run_command(emb_command)
        sg.load_embeddings(embed_file)  # update embeddings with output from w2v
        sg.save_embeddings(init_file, 128)
        distance = core_movement(sg.embeddings)
        prev_embeds = store_core_embeds(sg.embeddings)

Current time period: (1/9)


	Loaded embeddings. Dimensions: (14, 128)
Current time period: (2/9)
	Walking...
	Updating embeddings...


	Merging updated embeddings
	Loaded embeddings. Dimensions: (31, 128)
	Distance this iteration: 14.3636
Current time period: (3/9)
	Walking...
	Updating embeddings...


	Merging updated embeddings
	Loaded embeddings. Dimensions: (83, 128)
	Distance this iteration: 2.4571
Current time period: (4/9)
	Walking...


	Updating embeddings...


...

	Updating embeddings...


	Merging updated embeddings
	Loaded embeddings. Dimensions: (148, 128)
	Distance this iteration: 2.1429
Current time period: (7/9)


	Testing...
	Found 1096 new edges out of 2192 total pairs
	Using the pairs you provided...


	2192 pairs checked and 2192 pairs in dataframe


Prediction made.... Done


In [129]:
from sklearn.metrics import roc_auc_score
print("AUC: %.4f" % roc_auc_score(y_test, pred[:, 1]))
print("PR-AUC: %.4f" % average_precision_score(y_test, pred[:, 1], average='macro'))
print("NDCG: %.4f" % ndcg_at_k(pred[:,1], 100))
#bin = [int(x) for x in y_test]
#predicted =[]
#for i in range(0, pred.shape[0]):
#    predicted.append([pred[i, 0], pred[i, 1]])
#print("NDCG: %.4f" % ndcg_score(bin, predicted,5))

AUC: 0.6182
PR-AUC: 0.6100
NDCG: 0.7485


In [114]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [114]:
# for node in graph.nx_graph.nodes_iter():
#     print("%d")
#     for u, v in graph.nx_graph.edges(node):
#         print("\t%d, %d" % (u , v))